In [6]:
import requests
import pandas as pd
from datetime import datetime
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_market_news(api_key, limit=10):
    """
    Retrieve market news from Alpha Vantage API.

    Parameters:
    api_key (str): Alpha Vantage API key
    limit (int): Maximum number of news items to retrieve

    Returns:
    list: List of news items
    """
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'NEWS_SENTIMENT',
        'apikey': api_key,
        'limit': limit
    }
    
    try:
        logger.info("Fetching market news")
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        news_data = response.json()

        if 'Note' in news_data:
            logger.warning(f"API limit message: {news_data['Note']}")
            return []

        if 'feed' in news_data:
            logger.info(f"Retrieved {len(news_data['feed'])} news items")
            return news_data['feed']
        else:
            logger.warning("No news found in response")
            return []
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {str(e)}")
        return []
    except Exception as e:
        logger.error(f"Unexpected error occurred: {str(e)}")
        return []

def process_news_items(news_items):
    """
    Process news items into a structured DataFrame, including sentiment.

    Parameters:
    news_items (list): List of news items

    Returns:
    pd.DataFrame: DataFrame containing structured news data
    """
    try:
        processed_items = []
        for item in news_items:
            processed_item = {
                'ticker': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'title': item.get('title', ''),
                'summary': item.get('summary', ''),
                'publisher': item.get('source', ''),
                'link': item.get('url', ''),
                'publish_date': item.get('time_published', ''),
                'type': item.get('topics', ''),
                'related_tickers': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'overall_sentiment_score': item.get('overall_sentiment_score', ''),
                'overall_sentiment_label': item.get('overall_sentiment_label', '')
            }
            processed_items.append(processed_item)
        
        return pd.DataFrame(processed_items)
    except Exception as e:
        logger.error(f"Error processing news items: {str(e)}")
        return pd.DataFrame()

def save_news_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file.

    Parameters:
    df (pd.DataFrame): DataFrame containing news data
    output_dir (str): Directory to save the CSV file
    """
    try:
        if df.empty:
            logger.warning("No news data to save.")
            return None

        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Generate filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"market_news_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)

        # Save to CSV
        df.to_csv(filepath, index=False, encoding='utf-8')
        logger.info(f"News data successfully saved to: {filepath}")
        return filepath
    except Exception as e:
        logger.error(f"Error saving data to CSV: {str(e)}")
        return None

def main():
    """
    Main function to fetch, process, and save market news data locally.
    """
    # Configuration
    api_key = "YOUR_ALPHA_VANTAGE_API_KEY"  # Replace with your API key

    try:
        # Fetch market news
        news_items = get_market_news(api_key, limit=50)
        if not news_items:
            logger.warning("No news data retrieved.")
            return

        # Process news items
        news_df = process_news_items(news_items)

        # Save processed news data to CSV
        saved_file = save_news_to_csv(news_df)
        if saved_file:
            logger.info(f"News data saved successfully: {saved_file}")
        else:
            logger.warning("Failed to save news data.")
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")

if __name__ == "__main__":
    main()




2024-11-21 17:11:51,372 - INFO - Fetching market news
2024-11-21 17:11:51,709 - INFO - Retrieved 50 news items
2024-11-21 17:11:51,715 - INFO - News data successfully saved to: market_news\market_news_20241121_171151.csv
2024-11-21 17:11:51,716 - INFO - News data saved successfully: market_news\market_news_20241121_171151.csv


In [1]:
from datetime import datetime
import requests
import pandas as pd
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_market_news(api_key, limit=10):
    """
    Retrieve market news from Alpha Vantage API.

    Parameters:
    api_key (str): Alpha Vantage API key
    limit (int): Maximum number of news items to retrieve

    Returns:
    list: List of news items
    """
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'NEWS_SENTIMENT',
        'apikey': api_key,
        'limit': limit
    }
    
    try:
        logger.info("Fetching market news")
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        news_data = response.json()

        if 'Note' in news_data:
            logger.warning(f"API limit message: {news_data['Note']}")
            return []

        if 'feed' in news_data:
            logger.info(f"Retrieved {len(news_data['feed'])} news items")
            return news_data['feed']
        else:
            logger.warning("No news found in response")
            return []
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {str(e)}")
        return []
    except Exception as e:
        logger.error(f"Unexpected error occurred: {str(e)}")
        return []

def process_news_items(news_items):
    """
    Process news items into a structured DataFrame, including sentiment.

    Parameters:
    news_items (list): List of news items

    Returns:
    pd.DataFrame: DataFrame containing structured news data
    """
    try:
        processed_items = []
        for item in news_items:
            # Convert publish_date format
            raw_date = item.get('time_published', '')
            try:
                formatted_date = datetime.strptime(raw_date, "%Y%m%dT%H%M%S").strftime("%m/%d/%Y %H:%M")
            except ValueError:
                formatted_date = "Invalid Date"

            processed_item = {
                'ticker': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'title': item.get('title', ''),
                'summary': item.get('summary', ''),
                'publisher': item.get('source', ''),
                'link': item.get('url', ''),
                'publish_date': formatted_date,
                'type': item.get('topics', ''),
                'related_tickers': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'source': 'Alpha',
                'overall_sentiment_score': item.get('overall_sentiment_score', ''),
                'overall_sentiment_label': item.get('overall_sentiment_label', '')
            }
            processed_items.append(processed_item)
        
        # Reorder columns
        column_order = ['ticker', 'title', 'summary', 'publisher', 'link', 'publish_date', 'type', 
                        'related_tickers', 'source', 'overall_sentiment_score', 'overall_sentiment_label']
        return pd.DataFrame(processed_items)[column_order]
    except Exception as e:
        logger.error(f"Error processing news items: {str(e)}")
        return pd.DataFrame()

def save_news_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file.

    Parameters:
    df (pd.DataFrame): DataFrame containing news data
    output_dir (str): Directory to save the CSV file
    """
    try:
        if df.empty:
            logger.warning("No news data to save.")
            return None

        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Generate filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"market_news_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)

        # Save to CSV
        df.to_csv(filepath, index=False, encoding='utf-8')
        logger.info(f"News data successfully saved to: {filepath}")
        return filepath
    except Exception as e:
        logger.error(f"Error saving data to CSV: {str(e)}")
        return None

def main():
    """
    Main function to fetch, process, and save market news data locally.
    """
    # Configuration
    api_key = "YOUR_ALPHA_VANTAGE_API_KEY"  # Replace with your API key

    try:
        # Fetch market news
        news_items = get_market_news(api_key, limit=50)
        if not news_items:
            logger.warning("No news data retrieved.")
            return

        # Process news items
        news_df = process_news_items(news_items)

        # Save processed news data to CSV
        saved_file = save_news_to_csv(news_df)
        if saved_file:
            logger.info(f"News data saved successfully: {saved_file}")
        else:
            logger.warning("Failed to save news data.")
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")

if __name__ == "__main__":
    main()


2024-11-22 08:56:52,191 - INFO - Fetching market news
2024-11-22 08:56:52,600 - INFO - Retrieved 50 news items
2024-11-22 08:56:52,608 - INFO - News data successfully saved to: market_news\market_news_20241122_085652.csv
2024-11-22 08:56:52,609 - INFO - News data saved successfully: market_news\market_news_20241122_085652.csv


In [2]:
import requests
import pandas as pd
from datetime import datetime
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_market_news(api_key):
    """
    Retrieve market news from Alpha Vantage API.
    """
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'NEWS_SENTIMENT',
        'apikey': api_key
    }
    
    try:
        logger.info("Fetching market news")
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        news_data = response.json()

        if 'Note' in news_data:
            logger.warning(f"API limit message: {news_data['Note']}")
            return []

        if 'feed' in news_data:
            # Filter articles to include only today's articles
            today = datetime.now().strftime('%Y-%m-%d')
            filtered_news = [
                item for item in news_data['feed'] 
                if datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S").strftime('%Y-%m-%d') == today
            ]
            logger.info(f"Retrieved {len(filtered_news)} news items for today.")
            return filtered_news
        else:
            logger.warning("No news found in response")
            return []
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {str(e)}")
        return []
    except Exception as e:
        logger.error(f"Unexpected error occurred: {str(e)}")
        return []

def process_news_items(news_items):
    """
    Process news items into a structured DataFrame, including sentiment.
    """
    try:
        processed_items = []
        for item in news_items:
            # Convert publish_date format
            raw_date = item.get('time_published', '')
            try:
                formatted_date = datetime.strptime(raw_date, "%Y%m%dT%H%M%S").strftime("%m/%d/%Y %H:%M")
            except ValueError:
                formatted_date = "Invalid Date"

            processed_item = {
                'ticker': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'title': item.get('title', ''),
                'summary': item.get('summary', ''),
                'publisher': item.get('source', ''),
                'link': item.get('url', ''),
                'publish_date': formatted_date,
                'type': item.get('topics', ''),
                'related_tickers': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'source': 'Alpha',
                'overall_sentiment_score': item.get('overall_sentiment_score', ''),
                'overall_sentiment_label': item.get('overall_sentiment_label', '')
            }
            processed_items.append(processed_item)
        
        # Reorder columns
        column_order = ['ticker', 'title', 'summary', 'publisher', 'link', 'publish_date', 'type', 
                        'related_tickers', 'source', 'overall_sentiment_score', 'overall_sentiment_label']
        return pd.DataFrame(processed_items)[column_order]
    except Exception as e:
        logger.error(f"Error processing news items: {str(e)}")
        return pd.DataFrame()

def save_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a local CSV file.

    Parameters:
    df (pd.DataFrame): DataFrame containing news data
    output_dir (str): Directory to save the CSV file
    """
    try:
        if df.empty:
            logger.warning("No news data to save.")
            return None

        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Generate filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"market_news_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)

        # Save to CSV
        df.to_csv(filepath, index=False, encoding='utf-8')
        logger.info(f"News data successfully saved to: {filepath}")
        return filepath
    except Exception as e:
        logger.error(f"Error saving data to CSV: {str(e)}")
        return None

def main():
    """
    Main function to fetch, process, and save market news data locally.
    """
    # Replace this with your actual Alpha Vantage API key
    api_key = "FLGDYAANWX6EFL9P"

    try:
        # Fetch market news
        news_items = get_market_news(api_key)
        if not news_items:
            logger.warning("No news data retrieved.")
            return

        # Process news items
        news_df = process_news_items(news_items)

        # Save processed news data to a local CSV file
        saved_file = save_to_csv(news_df)
        if saved_file:
            logger.info(f"News data saved successfully: {saved_file}")
        else:
            logger.warning("Failed to save news data.")
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")

if __name__ == "__main__":
    main()


2024-11-22 09:46:44,993 - INFO - Fetching market news
2024-11-22 09:46:45,318 - INFO - Retrieved 50 news items for today.
2024-11-22 09:46:45,332 - INFO - News data successfully saved to: market_news\market_news_20241122_094645.csv
2024-11-22 09:46:45,332 - INFO - News data saved successfully: market_news\market_news_20241122_094645.csv
